In [1]:
! pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 5.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import pingouin as pg
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency

# Funções geradas na análise exploratória


In [ ]:
df_merged = pd.read_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_data_merged.csv')

### Entendendo cada variável:

Primeiro é importante entender como as variáveis são definidas pela estatística:

**Qualitativas (categóricas)** → utilizadas para descrever ou categorizar, não podem ser quantificadas. Podem ser subdivididas por:

-   **Ordinal**: quando os valores têm um certo tipo de hierarquia
    
    -   Exemplo: nível de escolaridade, dificuldade de uma tarefa
        
    -   Importante destacar que variáveis numéricas podem ser categorizadas e se tornarem qualitativas ordinais (ex: renda baixa, média ou alta per capita)
        
-   **Nominal**: os valores não têm hierarquia
    
    -   Exemplo: cor dos olhos, estado civil, nacionalidade
        

**Quantitativas (numéricas)** → utilizadas para representar quantidades ou medidas, podem ser expressas numericamente. Podem ser subdivididas por:

-   **Discretas**: assumem apenas valores inteiros e contáveis
    
    -   Exemplo: número de filhos, quantidade de carros em uma garagem
    - Em diversos casos, variáveis contínuas podem ser discretizadas
        
-   **Contínuas**: podem assumir qualquer valor dentro de um intervalo, inclusive frações e decimais
    
    -   Exemplo: altura, peso, temperatura corporal


In [ ]:
df_merged.info()

### Quantitativas (numéricas) → Discretas ou contínuas

-   **Colunas numéricas contínuas:** open, high, low, close, adj close, dividend, market cap (usd), pe ratio, eps (usd), previous close (usd), day low (usd), day high (usd)
    
-   **Colunas numéricas discretas:** volume
    
-   **Colunas categóricas nominais:** id, date, ticker, sector, industry, exchange, next earnings date,  Company
    

Importante destacar que o dataframe não apresenta todos os tipos de variáveis (por exemplo, categórica ordinal que tem uma ordem ou classificação entre as categorias)


In [ ]:
date_columns = ["Date", "Next Earnings Date"]

df_merged[date_columns] = df_merged[date_columns].apply(
    lambda col: pd.to_datetime(col, format="%d/%m/%Y", errors='coerce')
)

In [ ]:
df_merged["timestamp_date"] = df_merged["Date"].astype(np.int64) // 10 ** 9
df_merged["timestamp_next"] = df_merged["Next Earnings Date"].astype(np.int64) // 10 ** 9

In [ ]:
# Identificar colunas numéricas para análise de correlação
numerical_cols = df_merged.select_dtypes(include=np.number).columns.tolist()

In [ ]:
# Remover colunas que não são relevantes para a análise
columns_to_exclude_from_corr = ['ID'] # Coluna ID não é interessante
numerical_cols_for_corr = [col for col in numerical_cols if col not in columns_to_exclude_from_corr]

### Gerando matriz de correlação de pearson entre variáveis numericas
- Correlação entre váriaves: nível que um coluna se relaciona com outra de forma linear.
  Exemplo: é tirada uma amostra de peso e altura em uma sala, após a captação dos dados corretos é possível entender se peso tem correlação com altura. Quanto de forma linear peso interfere na altura da pessoa?

## Correlação de Pearson (r)

A fórmula do coeficiente de correlação de Pearson é:

$$
r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}
         {\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2} \cdot \sqrt{\sum_{i=1}^{n} (y_i - \bar{y})^2}}
$$
### Onde:
- r: coeficiente de correlação de Pearson
- xᵢ: valor da variável X na posição i (peso de uma pessoa da sala na linha i)
- yᵢ: valor da variável Y na posição i (altura de uma pessoa da sala na linha i)
- x̄: média dos valores da variável X (média do peso)
- ȳ: média dos valores da variável Y (média da altura)
- Σ: somatório para i = 1 até n
- n: número de pares de dados (número amostra)

In [ ]:


corr = df_merged[numerical_cols_for_corr].corr()

fig = px.imshow(
    corr,
    text_auto='.4f',          # Mostra os valores com 2 casas decimais
    aspect='auto',            # Ajusta proporção
    title='Matriz de Correlação',
    color_continuous_scale='RdBu'
)

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=100, r=100, t=100, b=100),
)

fig.show()


### Teste de hipótese para correlação (\( r \))
Após gerar o mapa de calor, realizei um teste de hipótese para verificar a significância estatística dos coeficientes de correlação observados. O teste serve como um detector de confiança para entender os padrões vistos. Será que esse valor é real ou uma coincidência?

O teste segue a seguinte estrutura:

- **H₀**: Hipótese nula \( r = 0 \)→ Sem correlação linear  real, o valor encontrado foi por acaso
- **H₁**: Hipótese alternativa ( r≠0 ) → Correlação linear presente  
- Utiliza-se a distribuição de t de Student como uma régua para medir as chances. Se um valor for raro (menor que α) rejeitamos H₀ e conclui-se que tem um padrão que merece uma investigação mais profunda.

A estatística de teste é calculada por:

$$
t = r \cdot \sqrt{\frac{n - 2}{1 - r^2}}
$$

Onde:  
- \( r \): coeficiente de correlação amostral  
- \( n \): número de observações  

Rejeita-se a hipótese nula quando o **valor absoluto da estatística t calculada** excede o **valor crítico da distribuição t** no nível de significância α/2 (já que é bicaudal) com n−2 graus de liberdade.

In [ ]:
pvals = corr.corr(method = lambda x, y: pg.corr(x, y)["p-val"].iloc[0])
np.fill_diagonal(pvals.values, np.nan)

fig = px.imshow(
    pvals,
    text_auto='.4f',
    color_continuous_scale='RdBu_r'
    )

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=100, r=100, t=100, b=100),

)

fig

In [ ]:
def check_significance(pvals, alpha=0.05):
  n = len(pvals.columns)
  for i in range(n):
      for j in range(i + 1, n):  # Começa de i+1 para evitar diagonal e duplicatas
          if pvals.iloc[i, j] < alpha:
              var1 = pvals.columns[i]
              var2 = pvals.columns[j]
              print(f"Correlação significativa entre {var1} e {var2} (p-valor = {pvals.iloc[i, j]:.4f})")

check_significance(pvals)

### Entendendo relação entre categoricas e numericas
Para fazer isso, primeiro é necessário escolher

In [ ]:
#análise entre valores categoricos e numéricos
categorical_columns = df_merged.select_dtypes(exclude=[np.number, 'datetime']).columns.tolist()

In [ ]:
def anova(df, num_col, cat_col):
    # Pré-computa o agrupamento para reutilização
    grouped = df.groupby(cat_col)[num_col]

    groups = [group.values[~pd.isnull(group.values)] for _, group in grouped]

    f_val, p_val = stats.f_oneway(*groups)
    return p_val

results = []

def testing_anova(df, categorical_columns, numerical_cols):
    for cat_col in categorical_columns:
        for num_col in numerical_cols:
            p_val = anova(df, num_col, cat_col)
            if p_val is not None and p_val < 0.05:
                print(f"Significativo: {num_col} e {cat_col} (p-valor={p_val:.6f})")
                results.append((cat_col, num_col, p_val))


testing_anova(df_merged, categorical_columns, numerical_cols)

## Fórmula do V de Cramér

$$
V = \sqrt{ \frac{\chi^2}{n \cdot (\min(r - 1, k - 1))} }
$$


In [ ]:
# Matriz para armazenar resultados
cramers_matrix = pd.DataFrame(
    np.nan,
    index=categorical_columns,
    columns=categorical_columns
)

# Preencher diagonal com 1 (correlação perfeita consigo mesma)
np.fill_diagonal(cramers_matrix.values, 1.0)

# Função para calcular V de Cramér
def cramers_v(contingency_table):
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    phi2 = chi2 / n
    r, k = contingency_table.shape
    return np.sqrt(phi2 / min(k-1, r-1))

# Calcular associação para cada par
for i, col1 in enumerate(categorical_columns):
    for j, col2 in enumerate(categorical_columns):
        if i < j:  # Evitar cálculos duplicados
            contingency = pd.crosstab(
                df_merged[col1],
                df_merged[col2]
            )
            cv = cramers_v(contingency.values)
            cramers_matrix.loc[col1, col2] = cv
            cramers_matrix.loc[col2, col1] = cv  # Matriz simétrica

fig = px.imshow(
    cramers_matrix,
    text_auto='.4f',
    color_continuous_scale='RdBu_r'
    )

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=100, r=100, t=100, b=100),

)

fig

### Análise de relações entre variáveis categóricas:
- Valores confirmam que não existem erros entre as variáves
- Setor e industrias estão fortemente relacionados, alto sentido lógico
- Exchange não apresenta nenhuma relação relevante com as outras variáveis categóricas

# **Análise Bivariada**


## **Análise de Dados Financeiros de Empresas**

**Conversão para Numérico:**

As colunas Adj Close (Preço de Fechamento Ajustado), Market Cap (USD) (Capitalização de Mercado em USD), PE Ratio (Índice Preço/Lucro) e EPS (USD) (Lucro Por Ação em USD) do DataFrame df_merged são convertidas para o tipo de dado numérico usando pd.to_numeric().


**Estatísticas Descritivas:**

O método .describe() é chamado para as colunas selecionadas. Ele calcula um resumo de estatísticas descritivas para cada uma dessas colunas numéricas.


In [ ]:
# 1. Preço Ajustado
df_merged['Adj Close'] = pd.to_numeric(df_merged['Adj Close'], errors='coerce')

# 2. Market Cap
df_merged['Market Cap (USD)'] = pd.to_numeric(df_merged['Market Cap (USD)'], errors='coerce')

# 3. Indicadores financeiros
df_merged['PE Ratio'] = pd.to_numeric(df_merged['PE Ratio'], errors='coerce')
df_merged['EPS (USD)'] = pd.to_numeric(df_merged['EPS (USD)'], errors='coerce')

# Exibir descrição geral dessas variáveis
print(df_merged[['Adj Close', 'Market Cap (USD)', 'PE Ratio', 'EPS (USD)']].describe())

**Explicação do Output:**

**count:** Número de observações não nulas.

- Adj Close, Market Cap (USD), PE Ratio têm 200.000 entradas.
- E- PS (USD) tem 199.918 entradas, indicando 82 valores ausentes (NaN), provavelmente devido à coerção de erros durante a conversão para numérico.

- mean: Média aritmética.
- std: Desvio padrão, uma medida de dispersão dos dados em torno da média.
- min: Valor mínimo observado.
- 25% (1º Quartil): Valor abaixo do qual 25% dos dados se encontram.
- 50% (Mediana ou 2º Quartil): Valor central, abaixo do qual 50% dos dados se encontram.
- 75% (3º Quartil): Valor abaixo do qual 75% dos dados se encontram.
- max: Valor máximo observado.
Observações Específicas do Output:

- Adj Close: A média (775.46) e a mediana (775.40) são muito próximas, sugerindo uma distribuição relativamente simétrica. O desvio padrão (418.04) indica uma variabilidade considerável nos preços.
- Market Cap (USD): A média (2.739×10
19
 ) é ordens de magnitude maior que a mediana (1.058×10
12
 , ou 1.058 trilhões de USD). Isso indica uma forte assimetria positiva (skewness), onde poucos valores extremamente altos puxam a média para cima. A mediana é uma medida de tendência central mais representativa neste caso. O desvio padrão também é gigantesco.
- PE Ratio: Média (52.55) e mediana (52.57) são próximas. O intervalo de 5 a 100 pode indicar que os dados foram filtrados ou os P/Es extremos foram limitados (capped).
- EPS (USD): Média (7.50) e mediana (7.48) próximas. A presença de um min de -5.00 indica que algumas empresas no conjunto de dados tiveram prejuízo. O desvio padrão (7.22) é quase do tamanho da média, mostrando alta variabilidade.

## **Estatísticas Agrupadas por Setor**
**Explicação do Código:**

- **Agrupamento:** O DataFrame df_merged é agrupado pela coluna Sector usando .groupby('Sector').
- **Seleção de Colunas e Agregação:** Para cada setor, são selecionadas as colunas Adj Close, Market Cap (USD), PE Ratio e EPS (USD).
- **Cálculo de Estatísticas:** A função .agg() é usada para calcular múltiplas estatísticas agregadas (mean, median, std, count) para as colunas selecionadas dentro de cada grupo (setor).
- **Visualização:** O DataFrame resultante sector_stats é impresso.

In [ ]:
# Agrupando por Setor
sector_stats = df_merged.groupby('Sector')[['Adj Close', 'Market Cap (USD)', 'PE Ratio', 'EPS (USD)']].agg(['mean', 'median', 'std', 'count'])

# Visualizar estatísticas por setor
print(sector_stats)

**Explicação do Output:**
O output mostra as estatísticas (mean, median, std, count) para cada uma das quatro variáveis financeiras, agora segmentadas por Sector.

- **count:**
O setor de Technology (Tecnologia) tem a maior contagem (133.649), sendo o mais representado.
Communication Services (Serviços de Comunicação) e Consumer Discretionary (Consumo Discricionário) têm contagens semelhantes (cerca de 33.000 cada).
- **Adj Close:**
As médias e medianas do preço ajustado são notavelmente similares entre os três setores (todos em torno de 770-777). Os desvios padrão também são consistentes.
- **Market Cap (USD):**
A média continua inflada por outliers em todos os setores.
As medianas do Market Cap são muito próximas para todos os setores (cerca de 1.06×10
12
  USD), sugerindo que o "tamanho típico" da empresa (pela mediana) é bastante homogêneo entre os setores presentes.
- **PE Ratio:**
Médias e medianas do P/L são muito semelhantes entre os setores (todos em torno de 52). Os desvios padrão também são consistentes.
- **EPS (USD):**
Médias e medianas do LPA também são muito parecidas entre os setores (em torno de 7.4-7.6). Os desvios padrão são consistentes.
A contagem para EPS (USD) é ligeiramente menor em cada setor, refletindo os valores ausentes observados na análise geral.

**Observação Chave**: Há uma notável consistência nas métricas de tendência central (média/mediana) e dispersão (desvio padrão) para Adj Close, PE Ratio, e EPS (USD) entre os diferentes setores. Apenas o Market Cap (USD) mostra grande variação interna (devido à assimetria), mas as medianas entre os setores são parecidas.

## **Estatísticas Agrupadas por Indústria**

**Explicação do Código:**

- **Agrupamento:** Similar à célula anterior, mas agora o DataFrame df_merged é agrupado pela coluna Industry.
- **Seleção de Colunas e Agregação:** Para cada indústria, são selecionadas as mesmas quatro colunas financeiras.
- **Cálculo de Estatísticas:** A função .agg() calcula mean, median, std e count para as colunas selecionadas dentro de cada grupo (indústria).
Visualização: O DataFrame resultante industry_stats é impresso.

In [ ]:
# Agrupando por Indústria
industry_stats = df_merged.groupby('Industry')[['Adj Close', 'Market Cap (USD)', 'PE Ratio', 'EPS (USD)']].agg(['mean', 'median', 'std', 'count'])

# Visualizar estatísticas por indústria
print(industry_stats)

**Explicação do Output:**
O output detalha as estatísticas para as mesmas quatro variáveis financeiras, mas agora em um nível mais granular, por Industry.

- **count:**
Software é a indústria com maior número de entradas (80.227).
Interactive Media & Services (33.254) e Semiconductors (20.082) também são significativas. As outras indústrias têm contagens menores.
- **Adj Close:**
As médias e medianas do preço ajustado continuam muito consistentes entre as diversas indústrias (geralmente entre 767 e 784). Os desvios padrão também são semelhantes.
- **Market Cap (USD):**
A média permanece distorcida por valores extremos em todas as indústrias.
As medianas do Market Cap são, novamente, notavelmente próximas para todas as indústrias (a maioria entre 1.05×10
12
  e 1.08×10
12
  USD).
PE Ratio:
Médias e medianas do P/L são muito semelhantes entre as indústrias (geralmente entre 51.7 e 53.2). Os desvios padrão são consistentes.
EPS (USD):
Médias e medianas do LPA também são muito parecidas entre as indústrias (geralmente entre 7.3 e 7.6). Os desvios padrão são consistentes.
A contagem para EPS (USD) continua um pouco menor em cada indústria.
Observação Chave: A surpreendente homogeneidade observada no nível de setor para Adj Close, PE Ratio, e EPS (USD) persiste também no nível de indústria. Isso sugere que, dentro deste conjunto de dados, as características financeiras médias (exceto Market Cap, onde a mediana é mais estável) não diferem drasticamente entre as indústrias listadas.

# **Visualização de Dados Financeiros por Setor (Boxplots)**

**Como ler um Boxplot:**

A linha no meio da caixa representa a mediana `(o valor central, 50º percentil).`
A caixa em si representa o `intervalo interquartil (IQR)`, que contém `50% dos dados centrais (do 25º ao 75º percentil)`. A altura da caixa indica a dispersão dessa porção central dos dados.
As "hastes" que se estendem da caixa geralmente vão até 1,5 vezes o IQR a partir da borda da caixa, ou até o valor mínimo/máximo se estes estiverem dentro desse limite. Eles mostram a extensão dos dados "típicos".
Pontos individuais além das hastes são considerados `outliers.`


In [ ]:
# Ajustar o tamanho geral dos gráficos
plt.figure(figsize=(10, 6))

# Boxplot de Preço Ajustado por Setor
sns.boxplot(data=df_merged, x='Sector', y='Adj Close')
plt.title('Boxplot do Preço Ajustado por Setor')
plt.xticks(rotation=45)
plt.show()

## **Interpretação do Gráfico - Boxplot do Preço Ajustado (Adj Close)**

**Medianas:** As linhas medianas para os setores Technology, Consumer Discretionary e Communication Services estão em níveis muito semelhantes, visualmente localizadas um pouco abaixo de 800.


**Dispersão (IQR):** As alturas das caixas (IQRs) também são bastante parecidas entre os três setores, indicando uma dispersão similar nos 50% centrais dos dados de Adj Close.

**Hastes e Outliers:** As hastes se estendem de forma simétrica e similar para todos os setores, cobrindo uma faixa de aproximadamente 0 a 1500. Não há outliers visíveis plotados individualmente além das hastes neste gráfico, sugerindo que os valores mínimo e máximo estão dentro do alcance das hastes ou que os dados podem ter sido pré-processados.

**Conclusão:** A distribuição do Adj Close é notavelmente homogênea entre os três setores apresentados. Isso corrobora as estatísticas descritivas (médias e medianas muito próximas) que você obteve anteriormente.

In [ ]:
# Boxplot de Market Cap por Setor
sns.boxplot(data=df_merged, x='Sector', y='Market Cap (USD)')
plt.title('Boxplot do Market Cap por Setor')
plt.xticks(rotation=45)
plt.yscale('log')  # Pode ajudar se os valores forem muito altos
plt.show()

## **Interpretação do Gráfico - Boxplot Market Cap (USD):**

**Escala Logarítmica:** Note que o eixo y está em potências de 10 (ex: 10¹¹, 10¹³...)

**Medianas:** Apesar da vasta gama de valores, as medianas (as linhas dentro das caixas) para os três setores são muito próximas, localizando-se um pouco acima de 10¹²(1 trilhão de USD). Isso é consistente com as medianas vistas nas estatísticas descritivas.


**Dispersão (IQR):** As caixas em si são relativamente "curtas" e de altura semelhante na escala logarítmica, indicando que os 50% centrais dos dados, após a transformação logarítmica, têm uma dispersão comparável entre os setores.


**Hastes:** As hastes também mostram uma extensão similar na escala logarítmica.
Outliers: Este gráfico destaca visualmente a presença de numerosos e significativos outliers para todos os setores. Esses pontos se estendem muitas ordens de magnitude acima do corpo principal dos boxplots, chegando até perto de 10²¹21 ou 10²² USD. Isso confirma a extrema assimetria positiva que foi identificada nas estatísticas descritivas (onde a média era muito maior que a mediana).


**Conclusão:** Enquanto o Market Cap é bastante similar entre os setores, todos eles contêm empresas com valores de mercado extraordinariamente altos. A escala logarítmica é essencial para visualizar a distribuição principal e a extensão desses outliers.


In [ ]:
# Boxplot de PE Ratio por Setor
sns.boxplot(data=df_merged, x='Sector', y='PE Ratio')
plt.title('Boxplot do PE Ratio por Setor')
plt.xticks(rotation=45)
plt.show()

## **Interpretação do Gráfico - Boxplot PE Ratio**

**Medianas:** As medianas do PE Ratio são praticamente idênticas entre os três setores, localizadas um pouco acima de 50.

**Dispersão (IQR):** As alturas das caixas são muito semelhantes, indicando uma dispersão comparável nos 50% centrais dos dados de PE Ratio para cada setor.

**Hastes e Outliers:** As hastes se estendem de forma simétrica e parecem cobrir toda a faixa de dados de aproximadamente 5 a 100. Não há outliers individuais plotados. Isso sugere que os valores mínimo (5) e máximo (100) observados nas estatísticas descritivas podem representar limites nos dados, e todos os dados estão contidos dentro das hastes do boxplot.

**Conclusão:** A distribuição do PE Ratio é extremamente similar entre os setores. Os gráficos refletem a faixa de valores (5 a 100) que também foi vista nas estatísticas descritivas.

In [ ]:
# Boxplot de EPS por Setor
sns.boxplot(data=df_merged, x='Sector', y='EPS (USD)')
plt.title('Boxplot do EPS por Setor')
plt.xticks(rotation=45)
plt.show()

## **Interpretação do Gráfico - Boxplot EPS (USD):**

**Medianas:** As medianas do EPS (USD) são muito semelhantes entre os setores, localizadas em torno de 7.5.

**Dispersão (IQR):** As caixas têm alturas muito parecidas, indicando uma dispersão similar nos 50% centrais dos dados de EPS (USD).

**Hastes e Outliers:** As hastes se estendem de aproximadamente -5 a 20. A parte inferior da haste (e da caixa) abaixo de zero indica a presença de empresas com Lucro Por Ação negativo (prejuízo). Assim como no PE Ratio, não há outliers individuais visíveis, sugerindo que a faixa de -5 a 20 (vista nas estatísticas descritivas) pode representar limites nos dados.


**Conclusão:** A distribuição do EPS (USD) também é bastante homogênea entre os setores. A presença de valores negativos é claramente visível.

## **Conclusão Final sobre a Análise Financeiro em Relação ao Setor**

**Homogeneidade** Para Adj Close, PE Ratio, e EPS (USD), as distribuições são notavelmente similares entre os setores Technology, Consumer Discretionary, e Communication Services.

**Market Cap:** O Market Cap (USD) é a exceção, mostrando medianas semelhantes, mas com uma vasta gama de valores e muitos outliers significativamente altos em todos os setores, o que é bem visualizado com o uso da escala logarítmica.

# **Visualização de Dados Financeiros por Indústria (Boxplots)**

In [ ]:
# Boxplot de Preço Ajustado por Indústria
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_merged, x='Industry', y='Adj Close')
plt.title('Boxplot do Preço Ajustado por Indústria')
plt.xticks(rotation=90)
plt.show()

## **Interpretação do Gráfico Preço de Fechamento (Adj Close) por Indústria:**

**Medianas:** Observando as linhas centrais dentro de cada caixa, as medianas do Adj Close são extraordinariamente consistentes entre todas as indústrias listadas (Software, Semiconductors, Interactive Media & Services, Automobiles, etc.), situando-se visualmente em torno de 750-800.

**Dispersão (IQR):** A altura das caixas (representando o intervalo interquartil) é também muito similar para todas as indústrias. Isso indica que os 50% centrais dos dados de Adj Close têm uma dispersão parecida, independentemente da indústria.

**Hastes e Outliers:** As hastes se estendem de forma similar para todas as indústrias, cobrindo a faixa aproximada de 0 a 1500. Não são visíveis outliers individuais além das hastes.

**Conclusão:** A homogeneidade na distribuição do Adj Close que foi observada na análise por setor se mantém de forma iguais.

In [ ]:
# Boxplot de Market Cap por Indústria
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_merged, x='Industry', y='Market Cap (USD)')
plt.title('Boxplot do Market Cap por Indústria')
plt.xticks(rotation=90)
plt.yscale('log')
plt.show()

## **Interpretação do Gráfico Market Cap (USD) por Indústria:**

**Escala Logarítmica:** O eixo y está em potências de 10 (ex: 10
11¹10 , 10¹³).

**Medianas:** As medianas do Market Cap são muito consistentes entre todas as indústrias, localizadas um pouco acima de 10¹²(1 trilhão de USD).

**Dispersão (IQR):** As caixas são relativamente "baixas" e de altura semelhante na escala logarítmica, indicando que a dispersão dos 50% centrais dos dados (após transformação logarítmica) é comparável entre as indústrias.

**Outliers:** Para cada uma das indústrias listadas, há uma quantidade expressiva de outliers com valores de Market Cap muito elevados, estendendo-se até 10¹² USD ou mais.

**Conclusão:** O padrão de um "típico" Market Cap (mediana) similar, acompanhado por uma cauda longa de outliers de valores extremamente altos, é uma característica presente em todas as indústrias analisadas neste conjunto de dados. A escala logarítmica é fundamental para esta visualização.

In [ ]:
# Boxplot de PE Ratio por Indústria
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_merged, x='Industry', y='PE Ratio')
plt.title('Boxplot do PE Ratio por Indústria')
plt.xticks(rotation=90)
plt.show()

## **Interpretação do Gráfico PE Ratio por Indústria:**

**Medianas:** As medianas do PE Ratio são virtualmente idênticas para todas as indústrias, situando-se um pouco acima de 50.
Dispersão (IQR): As alturas das caixas são muito semelhantes entre as indústrias.

**Hastes e Outliers:** As hastes se estendem consistentemente de aproximadamente 5 a 100 para todas as indústrias. Não há outliers individuais visíveis. Isso reforça a ideia de que os dados de PE Ratio podem estar limitados a essa faixa (5-100), como observado nas estatísticas descritivas gerais.

**Conclusão:** A distribuição do PE Ratio é extremamente homogênea entre todas as indústrias apresentadas, refletindo a faixa de valores (5 a 100) que também foi observada nas análises anteriores.

In [ ]:
# Boxplot de EPS por Indústria
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_merged, x='Industry', y='EPS (USD)')
plt.title('Boxplot do EPS por Indústria')
plt.xticks(rotation=90)
plt.show()

## **Interpretação do Gráfico EPS (USD) por Indústria:**

**Medianas:** As medianas do EPS (USD) são muito semelhantes entre todas as indústrias, localizadas em torno de 7.5.

D**ispersão (IQR):** As caixas têm alturas muito parecidas, indicando uma dispersão similar nos 50% centrais dos dados de EPS (USD).

**Hastes e Outliers:** As hastes se estendem de aproximadamente -5 a 20 para todas as indústrias. A parte inferior das hastes e das caixas abaixo de zero demonstra claramente a presença de empresas com Lucro Por Ação negativo. Similarmente ao PE Ratio, não há outliers individuais visíveis, sugerindo que a faixa de -5 a 20 (vista nas estatísticas descritivas) pode representar limites impostos aos dados.

**Conclusão:** A distribuição do EPS (USD) também é bastante homogênea entre todas as indústrias analisadas, com a presença de valores negativos sendo uma característica comum.

## **Conclusão Final sobre a Análise Financeiro em Relação à Indùstria**

**Homogeneidade:** As distribuições de Adj Close, PE Ratio e EPS (USD) são similares, não apenas entre setores, mas também entre as diversas indústrias mais granulares. As medianas, IQRs e a extensão das hastes são quase idênticas.

**Market Cap Consistente em sua Assimetria:**  Para o Market Cap (USD), embora as medianas sejam semelhantes entre as indústrias (próximas a 1 trilhão de USD), todas as indústrias exibem uma grande quantidade de outliers com valores de capitalização de mercado extremamente altos.


# **Visualização das Médias de Métricas Financeiras por Setor**

In [ ]:
# Cálculo das médias por setor
sector_means = df_merged.groupby('Sector')[['Adj Close', 'Market Cap (USD)', 'PE Ratio', 'EPS (USD)']].mean().reset_index()

# Barplot de Preço Ajustado por Setor
sns.barplot(data=sector_means, x='Sector', y='Adj Close')
plt.title('Média do Preço Ajustado por Setor')
plt.xticks(rotation=45)
plt.show()

## **Análise do Gráfico sobre a Média do Preço Ajustado por Setor**

**Interpretação:** O gráfico de barras mostra a média do Adj Close (Preço de Fechamento Ajustado) para os setores Communication Services, Consumer Discretionary e Technology.

**Observações:** As alturas das barras para os três setores são visualmente quase idênticas, todas em torno de 770-780. Isso indica que a média do Preço Ajustado é muito similar entre esses setores.

In [ ]:
# Barplot de Market Cap por Setor
sns.barplot(data=sector_means, x='Sector', y='Market Cap (USD)')
plt.title('Média do Market Cap por Setor')
plt.xticks(rotation=45)
plt.yscale('log')
plt.show()

## **Análise do Gráfico sobre a média do Marketcap por Setor**

**Consumer Discretionary** apresenta a maior média de Market Cap (aproximadamente 2.81 x 10^19 USD).

**Technology** tem a segunda maior média (aproximadamente 2.75 x 10^19 USD)

**Communication Services** tem a menor média entre os três (aproximadamente 2.65 x 10^19 USD).

In [ ]:
# Barplot de PE Ratio por Setor
sns.barplot(data=sector_means, x='Sector', y='PE Ratio')
plt.title('Média do PE Ratio por Setor')
plt.xticks(rotation=45)
plt.show()

## **Análise do Gráfico sobre a Média do PE Ratio por Setor**

As alturas das barras são praticamente idênticas para os três setores, todas em torno de 52-53. Isso indica que a média do PE Ratio é muito similar entre Communication Services, Consumer Discretionary e Technology

In [ ]:
# Barplot de EPS por Setor
sns.barplot(data=sector_means, x='Sector', y='EPS (USD)')
plt.title('Média do EPS por Setor')
plt.xticks(rotation=45)
plt.show()

## **Análise do Gráfico sobre a Média do EPS por Setor**

As alturas das barras são muito semelhantes para os três setores. Communication Services e Technology têm barras ligeiramente mais altas (em torno de 7.5-7.6) do que Consumer Discretionary (em torno de 7.4). As diferenças são mínimas.

## **Visualização da Distribuição de Métricas Financeiras por Setor (Violin Plots)**

Um `"violin plot" (gráfico de violino)` é uma representação gráfica da distribuição de dados numéricos, semelhante a um boxplot, mas com a adição de um gráfico de densidade de kernel rotacionado em cada lado. Esta visualização ajuda a entender a distribuição dos dados, incluindo a forma, a assimetria e a presença de múltiplos picos

In [ ]:
# Violin plot de Preço Ajustado por Setor
sns.violinplot(data=df_merged, x='Sector', y='Adj Close')
plt.title('Violin Plot do Preço Ajustado por Setor')
plt.xticks(rotation=45)
plt.show()

**Interpretação:** O gráfico mostra a distribuição da densidade do Adj Close (Preço de Fechamento Ajustado) para os setores Technology, Consumer Discretionary e Communication Services.

**Forma do Violino:** Os violinos para os três setores são praticamente idênticos em forma e tamanho. Eles são mais largos na região central (em torno de 400-1100) e se afunilam nas extremidades (próximo de 0 e acima de 1500). A forma sugere uma concentração de dados na faixa intermediária, com menos dados nos extremos. A simetria da forma em torno da mediana é notável.

**Boxplot Interno:** O pequeno boxplot dentro de cada violino confirma as observações dos boxplots anteriores: medianas (linha branca no centro do boxplot interno) muito próximas entre os setores (em torno de 775) e IQRs (altura da caixa interna) similares.

**Comparação com Boxplot:** Enquanto o boxplot resumia os quartis, o violin plot adiciona a percepção da densidade. Aqui, ele confirma que não apenas os quartis são semelhantes, mas toda a forma da distribuição dos preços ajustados é muito consistente entre os setores.

**Conclusão:** A homogeneidade da distribuição do Adj Close entre os setores é fortemente evidenciada, tanto em termos de resumo estatístico (boxplot interno) quanto de forma de densidade.

In [ ]:
# Violin plot de Market Cap por Setor
sns.violinplot(data=df_merged, x='Sector', y='Market Cap (USD)')
plt.title('Violin Plot do Market Cap por Setor')
plt.xticks(rotation=45)
plt.yscale('log')
plt.show()

**Interpretação:** Este violin plot exibe a distribuição da densidade do Market Cap (USD) por setor, com o eixo y em escala logarítmica.
Escala Logarítmica: Essencial para esta variável. O eixo y varia de 10¹¹. 10²¹...)

**Forma do Violino:** Os violinos têm uma forma muito característica devido à extrema assimetria dos dados de Market Cap. Eles são muito largos na base, em torno de 10¹² (1 trilhão de USD) na escala logarítmica, indicando uma alta concentração de empresas com Market Cap nessa faixa. Acima disso, os violinos se afunilam drasticamente, transformando-se em uma "haste" ou "cauda" muito longa e fina que se estende até valores extremamente altos (próximo a 10²¹
 ). Esta cauda fina representa os outliers de Market Cap muito elevado, que são poucos em número (baixa densidade), mas cobrem uma vasta gama de valores.
Boxplot Interno: O boxplot interno está comprimido na parte inferior do violino (em torno de 10¹²), com a mediana claramente visível e um IQR relativamente pequeno nessa escala.

**Comparação com Boxplot:** O violin plot visualiza de forma ainda mais dramática a natureza da distribuição do Market Cap: a maioria das empresas concentradas em valores "menores" (na casa dos trilhões, na escala log) e uma minoria de empresas com valores astronomicamente altos. O boxplot mostrava os outliers como pontos individuais; o violino mostra a continuidade da distribuição (embora com densidade muito baixa) até esses valores extremos.

**Conclusão:** A forma dos violinos para o Market Cap é similar entre os setores, todos mostrando uma base larga e uma cauda superior longa e fina, destacando a concentração na faixa inferior e a presença de outliers significativos.

In [ ]:
# Violin plot de PE Ratio por Setor
sns.violinplot(data=df_merged, x='Sector', y='PE Ratio')
plt.title('Violin Plot do PE Ratio por Setor')
plt.xticks(rotation=45)
plt.show()

**Interpretação:** O gráfico mostra a distribuição da densidade do PE Ratio por setor.

**Forma do Violino:** Os violinos para os três setores são, novamente, quase idênticos. Eles são mais largos na região central e se afunilam nas extremidades, que correspondem aos valores mínimo (5) e máximo (100) observados anteriormente. A forma sugere uma distribuição um tanto uniforme na parte central, com a maior densidade em torno da mediana.
Boxplot Interno: Confirma a mediana em torno de 52-53 e o IQR já visto nos boxplots simples.

**Comparação com Boxplot:** O boxplot mostrava os limites (5 e 100) e os quartis. O violin plot adiciona a informação de que a densidade dos dados é relativamente bem distribuída entre esses limites, com uma concentração maior no meio. A ausência de "caudas" longas além de 5 e 100 reforça a ideia de que os dados para PE Ratio estão contidos nessa faixa.

**Conclusão:** A distribuição da densidade do PE Ratio é muito similar entre os setores, provavelmente refletindo dados que foram limitados ou filtrados na faixa de 5 a 100.


In [ ]:
# Violin plot de EPS por Setor
sns.violinplot(data=df_merged, x='Sector', y='EPS (USD)')
plt.title('Violin Plot do EPS por Setor')
plt.xticks(rotation=45)
plt.show()

**Interpretação:** Este violin plot mostra a distribuição da densidade do EPS (USD) por setor.

**Forma do Violino:** A forma dos violinos é notavelmente similar entre os três setores. Eles são mais largos na região central (em torno de 1 a 14) e se afunilam nas extremidades, que correspondem aos valores mínimo (-5) e máximo (20). A parte inferior do violino, estendendo-se abaixo de zero, indica a densidade de empresas com EPS negativo.

**Boxplot Interno:** O boxplot interno mostra a mediana em torno de 7.5 e o IQR correspondente, consistentes com as análises anteriores.
Comparação com Boxplot: Adiciona à análise do boxplot a visualização da forma da densidade. Mostra que, embora haja EPS negativos, a maior concentração de dados (parte mais larga do violino) está nos valores positivos de EPS. A ausência de caudas longas além de -5 e 20 sugere, novamente, que os dados de EPS podem estar contidos nessa faixa.

**Conclusão:** A distribuição da densidade do EPS (USD) é muito consistente entre os setores, com uma forma que reflete a faixa de dados observada (-5 a 20) e uma maior concentração em torno da mediana positiva.

# **Resumo Geral dos Violin Plots**

**Homogeneidade Persistente:** Para Adj Close, PE Ratio e EPS (USD), não apenas os resumos estatísticos (como medianas e quartis) são semelhantes entre os setores, mas a forma completa da distribuição de densidade também é extraordinariamente consistente.

**Visualização da Densidade do Market Cap:** Para Market Cap (USD), o violin plot (com escala logarítmica) oferece uma visualização poderosa da extrema assimetria, mostrando a alta densidade de empresas em faixas de valor de mercado mais baixas (na casa dos trilhões de USD) e a baixíssima densidade (cauda longa e fina) de empresas com valores de mercado muito altos.

In [ ]:
df_merged["Year"] = df_merged["Date"].dt.year
df_merged["Month"] = df_merged["Date"].dt.month

In [ ]:
def plot_time_series(df, variable, time_period='M', group_by=None, title=None):

    temp_df = df.copy()

    # Criar coluna de período temporal
    if time_period == 'M':
        temp_df['Period'] = temp_df['Date'].dt.to_period("M").astype(str)
    elif time_period == 'Y':
        temp_df['Period'] = temp_df['Date'].dt.year.astype(str)
    else:
        raise ValueError("time_period deve ser 'M' ou 'Y'")

    # Agregar dados
    if group_by:
        aggregated = temp_df.groupby(['Period', group_by])[variable].mean().reset_index()
    else:
        aggregated = temp_df.groupby('Period')[variable].mean().reset_index()

    # Gerar título padrão se não fornecido
    if not title:
        title = f"Média de {variable} por {'Mês' if time_period=='M' else 'Ano'}"
        if group_by:
            title += f" e {group_by}"

    # Plotar
    if group_by:
        fig = px.line(aggregated, x='Period', y=variable, color=group_by, title=title)
    else:
        fig = px.line(aggregated, x='Period', y=variable, title=title)

    fig.show()
    return fig




def plot_timeline(df, variable, year_range=None, title=None):

    temp_df = df.copy()

    # Filtrar por ano se necessário
    if year_range:
        start, end = year_range
        temp_df = temp_df[temp_df['Year'].between(start, end)]

    # Agregar dados
    mean_by_company_year = temp_df.groupby(["Year", "Company"])[variable].mean().reset_index()
    max_per_year = mean_by_company_year.loc[mean_by_company_year.groupby("Year")[variable].idxmax()]

    # Preparar datas
    max_per_year["Start"] = pd.to_datetime(max_per_year["Year"].astype(int).astype(str) + "-01-01")
    max_per_year["End"] = pd.to_datetime(max_per_year["Year"].astype(int).astype(str) + "-12-31")

    # Gerar título padrão se não fornecido
    if not title:
        title = f"Empresa com Maior {variable} Médio por Ano"

    # Plotar
    fig = px.timeline(max_per_year,
                      x_start="Start",
                      x_end="End",
                      y="Company",
                      color="Company",
                      hover_data=["Year", variable],
                      title=title)

    fig.update_yaxes(autorange="reversed")
    fig.show()
    return fig

def plot_boxplot(df, x_var, y_var, color_var=None, filters=None, title=None):

    temp_df = df.copy()

    # Aplicar filtros
    if filters:
        for col, values in filters.items():
            temp_df = temp_df[temp_df[col].isin(values)]

    # Gerar título padrão se não fornecido
    if not title:
        title = f"Distribuição de {y_var} por {x_var}"
        if color_var:
            title += f" e {color_var}"

    # Plotar
    fig = px.box(temp_df,
                 x=x_var,
                 y=y_var,
                 color=color_var,
                 title=title)

    fig.show()
    return fig

def by_each_year(df, val, r=range(2023, 2026)):
    data_by_year = df.groupby(["Year", "Month"])[val].mean().reset_index()

    filtered_data = data_by_year[data_by_year["Year"].isin(r)]
    title = f"Entendendo média de {val} em cada ano separadamente"
    fig = px.line(filtered_data, x="Month", y=val, color="Year",
                  title=title,
                  markers=True)

    fig.show()



def sazonal_check(df, val, r= range(2021,2026)):

  data_close_month = df_merged.groupby("Month")[val].mean()



  fig = px.line(data_close_month, title=f"Media de {val} por mês")
  fig.show()

In [ ]:
# Analisar as 3 variáveis principais
variables = ['Close', 'Volume', 'Dividend']

for var in variables:
    # Série temporal mensal
    plot_time_series(df_merged, var, time_period='M', title=f"Série Temporal de {var} por Mês")

    sazonal_check(df_merged, var)

    by_each_year(df_merged, var)

    plot_boxplot(df_merged,
                     x_var='Month',
                     y_var=var,
                     color_var='Sector',
                     filters={'Year': [2023, 2025]},
                     title=f"Distribuição de {var} por Mês e Setor (2023-2024)")




In [ ]:
# valor medio de close em cada tipo de industria
fig = px.bar(df_merged.groupby("Industry")["Close"].mean().reset_index().sort_values(by="Close", ascending=False),
             x="Industry",
             y="Close",
             color= "Close")
#y começando com 700
fig.update_layout(yaxis_range=[700, 800])


fig

In [ ]:
print(anova(df_merged, "Close", "Year"))
print(anova(df_merged, "Close", "Month"))


### Correlações lineares fortes encontradas anteriores, para entender o que pode inflênciar o valor de Close:

- Correlação significativa entre Open e Close (p-valor = 0.0121)
- Correlação significativa entre High e Close (p-valor = 0.0022)
- Correlação significativa entre Low e Close (p-valor = 0.0023)

In [ ]:


def plot_normalized_lines(df, group_col, value_cols, title="Comparação Normalizada"):
    """
    Agrupa o DataFrame por uma coluna (ex: 'Year'), calcula a média das colunas numéricas fornecidas,
    normaliza os valores para a escala [0, 1] e plota as curvas no mesmo gráfico.

    Parâmetros:
    - df: DataFrame com os dados.
    - group_col: coluna para agrupar (ex: 'Year').
    - value_cols: lista de colunas numéricas a serem comparadas (ex: ['Close', 'High', 'Low']).
    - title: título do gráfico.
    """
    # Agrupar por group_col e calcular média
    grouped = df.groupby(group_col)[value_cols].mean().reset_index()

    # Normalizar as colunas numéricas
    normalized = grouped.copy()
    for col in value_cols:
        min_val = grouped[col].min()
        max_val = grouped[col].max()
        normalized[col] = (grouped[col] - min_val) / (max_val - min_val)

    # Transformar para formato longo (long-form)
    long_df = normalized.melt(id_vars=group_col, value_vars=value_cols,
                              var_name="Variável", value_name="Valor Normalizado")

    # Plot
    fig = px.line(long_df, x=group_col, y="Valor Normalizado", color="Variável",
                  title=title, markers=True)
    fig.show()

# Exemplo com seu df_merged
plot_normalized_lines(df_merged, group_col="Year", value_cols=["Close", "High"],
                      title="Tendência Normalizada: Close, High e Low")


In [ ]:
def plot_avg_by_ticker(df, val_col="Close", year_min=2025, y_min=None, color_scale="Blues"):

    # Filtrar e agrupar
    por_ticker_df = (
        df[df["Year"] > year_min]
        .groupby("Ticker")[val_col]
        .mean()
        .reset_index()
        .sort_values(by=val_col, ascending=False)
    )

    # Criar gráfico
    fig = px.bar(
        por_ticker_df,
        x="Ticker",
        y=val_col,
        color=val_col,
        color_continuous_scale=color_scale,
        title=f"Média de {val_col} por Ticker (Year > {year_min})"
    )

    # Ajustar eixo Y, se necessário
    y_max = por_ticker_df[val_col].max()
    if y_min is not None:
        fig.update_layout(yaxis_range=[y_min, y_max + (0.05 * y_max)])

    fig.show()
plot_avg_by_ticker(df_merged, val_col="High", year_min=2023, y_min=1000)
plot_avg_by_ticker(df_merged, val_col="Volume", year_min=2023, y_min = 4 * 10**9)
plot_avg_by_ticker(df_merged, "Close", year_min=2023, y_min = 700)


In [ ]:
#checando significancia
print(anova(df_merged[df_merged["Year"]> 2023], "Close", "Ticker"))

In [ ]:
#entendendo variação de preço de INTU, UBER e TWTR de 2025 até 2025 de forma mensal:
# Filtro corrigido
agg = df_merged[
    (df_merged["Year"] > 2023) &
    (df_merged["Ticker"].isin(["INTU", "UBER", "TWTR"]))
]
agg = agg.groupby(["Year-Month", "Ticker"])["Close"].mean().reset_index()
fig = px.line(agg, x="Year-Month", y="Close", color="Ticker")
fig.show()

In [ ]:
df_agg = df_merged[df_merged["Year"] > 2023].groupby(["Sector", "Ticker", "Industry"])["Volume"].sum().reset_index()
fig = px.sunburst(
    df_agg,
    path=['Sector','Industry', 'Ticker'],  # Hierarquia: Sector -> Ticker
    values='Volume',             # Valores baseados no Volume
    branchvalues='total',         # Garante que os pais mostrem soma dos filhos
    hover_data={'Volume': ':.2f'},# Formata hover
    labels={'Volume': 'Volume'},
)
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.show()

In [ ]:
volume_sector_year = df_merged.groupby(["Year", "Sector"])["Volume"].mean().reset_index()

fig = px.line(volume_sector_year, x="Year", y="Volume", color="Sector",
              title="Média do Fechamento Mensal por Ano",
              markers=True)

fig.show()

In [ ]:
testing_anova(volume_sector_year, ["Sector"], ["Volume"])

In [ ]:
volume_sector_year = df_merged.groupby(["Year", "Sector"])["Dividend"].mean().reset_index()

fig = px.line(volume_sector_year, x="Year", y="Dividend", color="Sector",
              title="Média de dividendo por ano",
              markers=True)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

def plot_volume_dividends_by_sector(df_merged):
    """
    Agrupa os dados por setor, ano e mês, calcula agregações relevantes e
    gera um gráfico de dispersão com Plotly Express mostrando a relação entre
    volume médio e dividendos totais por setor.

    Parâmetros:
    - df_merged: DataFrame contendo as colunas 'Sector', 'Year', 'Month', 'Volume', 'Dividend' e 'Market Cap (USD)'.

    Retorna:
    - fig: Objeto de figura do Plotly.
    """
    # Agregar dados
    df_agg = df_merged.groupby(['Sector', 'Year', 'Month'], as_index=False).agg({
        'Volume': 'mean',
        'Dividend': 'sum',
        'Market Cap (USD)': 'mean'
    })

    # Criar coluna "Year-Month"
    df_agg['Year-Month'] = df_agg['Year'].astype(str) + '-' + df_agg['Month'].astype(str).str.zfill(2)

    # Gerar gráfico
    fig = px.scatter(
        df_agg,
        x='Year-Month',
        y='Volume',
        size='Dividend',
        color='Sector',
        hover_name='Sector',
        size_max=30,
        title='Relação entre Volume Médio e Dividendos por Setor',
        labels={'Volume': 'Volume Médio', 'Year-Month': 'Período', 'Dividend': 'Dividendos Totais'},
        template='plotly_white'
    )

    fig.update_layout(hovermode='x unified')
    return fig

fig = plot_volume_dividends_by_sector(df_merged)
fig.show()



In [ ]:
def calcular_saude_empresa(df, pesos=None, plot=False):
    import numpy as np
    import pandas as pd
    from scipy.stats import zscore
    import matplotlib.pyplot as plt

    if pesos is None:
        pesos = {
            'crescimento': 0.25,
            'dividendo': 0.25,
            'volume': 0.20,
            'market_cap': 0.15,
            'pe_ratio': 0.15,
            'volatilidade': 0.10
        }

    df_sorted = df.sort_values(by=['Company', 'Date']).copy()
    df_sorted['Ano'] = pd.to_datetime(df_sorted['Date']).dt.year

    resultados_raw = []

    for company, grupo in df_sorted.groupby('Company'):
        try:
            ano_inicio = grupo['Ano'].min()
            ano_fim = grupo['Ano'].max()
            media_inicio = grupo[grupo['Ano'] == ano_inicio]['Close'].mean()
            media_fim = grupo[grupo['Ano'] == ano_fim]['Close'].mean()
            crescimento = (media_fim - media_inicio) / media_inicio if media_inicio != 0 else 0
        except:
            crescimento = 0

        volatilidade = np.mean(grupo['High'] - grupo['Low']) / grupo['Open'].mean() if grupo['Open'].mean() != 0 else 0
        dividendo = grupo['Dividend'].mean()
        volume = grupo['Volume'].mean()
        market_cap = grupo['Market Cap (USD)'].mean()
        pe = grupo['PE Ratio'].mean()
        pe_desvio = abs(pe - 20)

        resultados_raw.append({
            'Company': company,
            'Crescimento': crescimento,
            'Volatilidade': 1 - volatilidade,
            'Dividendo': dividendo,
            'Volume': volume,
            'Market Cap': market_cap,
            'PE Ratio Score': 1 - (pe_desvio / 40)
        })

    df_metricas = pd.DataFrame(resultados_raw).set_index('Company')

    # Padronização com z-score (algumas métricas têm direção invertida)
    for col in df_metricas.columns:
        if col in ['Volatilidade', 'PE Ratio Score']:  # já são "melhor quanto maior"
            df_metricas[col + '_z'] = zscore(df_metricas[col].fillna(0))
        else:
            df_metricas[col + '_z'] = zscore(df_metricas[col].fillna(0))

    # Escalonar z-scores para uma faixa 0–100 (usando percentis)
    df_scaled = df_metricas[[c for c in df_metricas.columns if c.endswith('_z')]].copy()
    df_scaled = df_scaled.apply(lambda x: 100 * (x.rank(pct=True)))

    # Score final ponderado
    df_scaled['Health Score'] = (
        df_scaled['Crescimento_z'] * pesos['crescimento'] +
        df_scaled['Dividendo_z'] * pesos['dividendo'] +
        df_scaled['Volume_z'] * pesos['volume'] +
        df_scaled['Market Cap_z'] * pesos['market_cap'] +
        df_scaled['PE Ratio Score_z'] * pesos['pe_ratio'] +
        df_scaled['Volatilidade_z'] * pesos['volatilidade']
    )

    # Pontos fortes e fracos
    metricas_reais = ['Crescimento', 'Dividendo', 'Volume', 'Market Cap', 'PE Ratio Score', 'Volatilidade']
    df_metricas['Health Score'] = df_scaled['Health Score']
    df_metricas['Ponto Forte'] = df_scaled[[
        m + '_z' for m in metricas_reais
    ]].idxmax(axis=1).str.replace('_z', '')
    df_metricas['Ponto Fraco'] = df_scaled[[
        m + '_z' for m in metricas_reais
    ]].idxmin(axis=1).str.replace('_z', '')

    # Gerar alertas
    def gerar_alertas(company):
        alertas = []

        if company in df_scaled['Crescimento_z'].nlargest(10).index:
            alertas.append('Crescimento entre os 10 maiores')
        elif company in df_scaled['Crescimento_z'].nsmallest(10).index:
            alertas.append('Crescimento entre os 10 menores')

        if company in df_scaled['Volatilidade_z'].nsmallest(10).index:
            alertas.append('Volatilidade entre as 10 maiores')

        if company in df_scaled['PE Ratio Score_z'].nsmallest(10).index:
            alertas.append('P/E Score entre os 10 piores')

        if company in df_scaled['Dividendo_z'].nlargest(10).index:
            alertas.append('Dividendos entre os 10 maiores')
        elif company in df_scaled['Dividendo_z'].nsmallest(10).index:
            alertas.append('Dividendos entre os 10 menores')

        if company in df_scaled['Volume_z'].nlargest(10).index:
            alertas.append('Volume entre os 10 maiores')
        elif company in df_scaled['Volume_z'].nsmallest(10).index:
            alertas.append('Volume entre os 10 menores')

        if company in df_scaled['Market Cap_z'].nlargest(10).index:
            alertas.append('Market Cap entre os 10 maiores')
        elif company in df_scaled['Market Cap_z'].nsmallest(10).index:
            alertas.append('Market Cap entre os 10 menores')

        return ', '.join(alertas) if alertas else 'OK'

    df_metricas['Alertas'] = df_metricas.index.map(gerar_alertas)

    # Distribuição
    if plot:
        df_metricas['Health Score'].hist(bins=50)
        plt.title('Distribuição dos Health Scores')
        plt.xlabel('Health Score')
        plt.ylabel('Nº de Empresas')
        plt.show()

    return df_metricas[['Health Score', 'Ponto Forte', 'Ponto Fraco', 'Alertas']].sort_values(by='Health Score', ascending=False).reset_index()




# Execução:
resultado = calcular_saude_empresa(df_merged)
resultado["Health Score"] = resultado["Health Score"].round(2)
resultado


 O que é Market Cap e por que analisar?
O Market Cap (Capitalização de Mercado) representa o valor de mercado de uma empresa, calculado por:

# **MarketCap = Preço da Ação X Quantidade total das ações**
Serve para classificar o tamanho da empresa:

- Small Caps: Pequenas empresas — maior risco e maior potencial de valorização.

- Mid Caps: Empresas médias — equilíbrio entre risco e retorno.

- Large Caps: Grandes empresas — mais estabilidade, menor risco e crescimento mais lento, porém mais seguro.

 Por que é importante?

1- Impacta diretamente a liquidez, risco e perfil do investidor.

2- Empresas maiores são mais resilientes em momentos de crise.

3- Empresas menores tendem a ser mais sensíveis ao cenário econômico, mas podem crescer mais rapidamente.

 **Distribuição dos Market Caps por Ticker**

Vamos observar o tamanho relativo das empresas presentes na base de dados.

In [ ]:
# Selecionando colunas relevantes
df_marketcap = df[['Ticker', 'Market Cap (USD)']].dropna()

# Convertendo Market Cap para numérico, caso não esteja
df_marketcap['Market Cap (USD)'] = pd.to_numeric(df_marketcap['Market Cap (USD)'], errors='coerce')

# Conferindo os dados
df_marketcap.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Agrupando Market Cap médio por ticker
marketcap_mean = df_marketcap.groupby('Ticker')['Market Cap (USD)'].mean().sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=marketcap_mean.values, y=marketcap_mean.index, palette="Blues_r")
plt.title('Market Cap Médio por Ticker', fontsize=14)
plt.xlabel('Market Cap (USD)')
plt.ylabel('Ticker')
plt.tight_layout()
plt.show()


###  Interpretação:

- É evidente que alguns tickers possuem um valor de mercado significativamente superior aos demais.
- **NVDA** e **META** estão entre os maiores Market Caps, representando grandes players do mercado.
- Outros tickers, como **WORK** e **INTU**, possuem valores bem inferiores, indicando empresas de menor porte.
- Este padrão revela a concentração de capital em poucas empresas, característica comum em mercados financeiros.


##   Distribuição do Market Cap por Ticker (Boxplot)

O boxplot permite visualizar a distribuição do Market Cap dentro de cada ticker, identificando **dispersões, outliers e simetrias** nos dados.


In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=df_marketcap, x='Market Cap (USD)', y='Ticker', palette="Set2")
plt.title('Distribuição do Market Cap por Ticker', fontsize=14)
plt.xlabel('Market Cap (USD)')
plt.ylabel('Ticker')
plt.tight_layout()
plt.show()


###  Interpretação:

- Observa-se uma grande **dispersão** no Market Cap de alguns tickers, como **NVDA**, com outliers representando períodos de pico ou quedas.
- Tickers menores apresentam caixas mais compactas, indicando menor variação nos seus valores de mercado.
- O boxplot confirma a forte assimetria na distribuição do Market Cap, com poucos tickers concentrando os valores mais altos.


## Densidade do Market Cap por Ticker (Violino)

O gráfico de violino combina boxplot com distribuição de densidade, oferecendo uma visão mais detalhada dos valores mais recorrentes de Market Cap para cada ticker.


In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(data=df_marketcap, x='Market Cap (USD)', y='Ticker', palette="pastel")
plt.title('Densidade do Market Cap por Ticker', fontsize=14)
plt.xlabel('Market Cap (USD)')
plt.ylabel('Ticker')
plt.tight_layout()
plt.show()


###  Interpretação:

- Empresas como **NVDA** possuem uma distribuição ampla, com concentração em determinados intervalos, mas também longas caudas.
- Empresas menores apresentam distribuições mais estreitas, indicando menos variabilidade no seu Market Cap.
- Esse tipo de gráfico ajuda a entender onde estão concentrados os valores mais comuns do Market Cap para cada ticker.


## Treemap — Participação Proporcional do Market Cap por Ticker

O treemap apresenta uma visão hierárquica e proporcional, onde o tamanho de cada bloco representa o Market Cap de cada ticker.


In [ ]:
import plotly.express as px

fig = px.treemap(df_marketcap, path=['Ticker'], values='Market Cap (USD)',
                  title='Participação Proporcional do Market Cap por Ticker',
                  color='Market Cap (USD)', color_continuous_scale='Viridis')
fig.show()


###Interpretação:

- O treemap evidencia claramente que a maior parte do Market Cap está concentrada em poucos tickers.
- Tickers como **NVDA** e **META** dominam a composição, refletindo seu peso no mercado.
- As empresas de menor Market Cap aparecem com blocos significativamente menores, reforçando o padrão de concentração.


#  Conclusão da Análise Bivariada(Ticker x MarketCap)

A análise da relação entre **Ticker** e **Market Cap (USD)** revela um mercado altamente concentrado, onde poucas empresas detêm a maior parte do valor total.

Empresas com Market Cap elevado tendem a ser mais estáveis e atrativas para perfis conservadores, enquanto empresas com Market Cap menor apresentam maiores riscos, mas também maior potencial de crescimento.

Este tipo de análise é essencial para entender a composição do portfólio e pode auxiliar na definição de estratégias de diversificação ou alocação de risco.


# **Perguntas**

1. Quais são os fatores que mais influenciam o preço de fechamento das ações ao longo do tempo?

2. Existe alguma associação entre características setoriais/industriais e o desempenho das ações?,
3. Como o volume de negociações e os dividendos distribuídos se comportam entre diferentes setores e ao longo do tempo?,
4. O que pode ser feito para otimizar as vendas?

## **2. Existe alguma associação entre características setoriais/industriais e o desempenho das ações?**

Ao realizar uma análise bivariada para investigar se existe uma associação significativa entre as características setoriais/industriais e o desempenho das ações, utilizando o Preço de Fechamento Ajustado (Adj Close) e a Capitalização de Mercado (Market Cap (USD)).

In [ ]:
df_plot = df_merged.dropna(subset=['Adj Close', 'Market Cap (USD)', 'Sector', 'Industry']).copy()
top_sectors = df_plot['Sector'].value_counts().nlargest(3).index
df_plot_filtered = df_plot[df_plot['Sector'].isin(top_sectors)]
unique_industries = df_plot['Industry'].unique()

# Estilo e paletas
sns.set(style="whitegrid", context="notebook", font_scale=1.2)
palette_sector = sns.color_palette("viridis", len(top_sectors))
palette_industry = sns.color_palette("viridis", len(unique_industries))

# Função para gerar violinplots
def plot_violin(x, y, data, palette, title, xlabel, ylabel, log_scale=False, rotate_xticks=False, fig_size=(10,5)):
    plt.figure(figsize=fig_size)
    sns.violinplot(x=x, y=y, data=data, inner='quartile', palette=palette)
    plt.title(title, fontsize=12)
    plt.xlabel(xlabel, fontsize=11)
    plt.ylabel(ylabel, fontsize=11)
    if log_scale:
        plt.yscale('log')
    if rotate_xticks:
        plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Gráficos
plot_violin(
    x='Sector', y='Adj Close', data=df_plot_filtered, palette=palette_sector,
    title='Preço de Fechamento Ajustado por Setor',
    xlabel='Setor', ylabel='Preço Ajustado (USD)'
)

plot_violin(
    x='Sector', y='Market Cap (USD)', data=df_plot_filtered, palette=palette_sector,
    title='Capitalização de Mercado por Setor (Log)',
    xlabel='Setor', ylabel='Cap. de Mercado (USD)',
    log_scale=True
)

plot_violin(
    x='Industry', y='Adj Close', data=df_plot, palette=palette_industry,
    title='Preço de Fechamento Ajustado por Indústria',
    xlabel='Indústria', ylabel='Preço Ajustado (USD)',
    rotate_xticks=True, fig_size=(12,6)
)

plot_violin(
    x='Industry', y='Market Cap (USD)', data=df_plot, palette=palette_industry,
    title='Capitalização de Mercado por Indústria (Log)',
    xlabel='Indústria', ylabel='Cap. de Mercado (USD)',
    log_scale=True, rotate_xticks=True, fig_size=(12,6)
)

Com base na análise visual dos gráficos de distribuição para as métricas de desempenho das ações (Adj Close e Market Cap) em relação aos Setores e Indústrias, a resposta é:

**Não**, a análise dos dados não revela uma associação forte ou uma dependência significativa entre as características setoriais/industriais e o desempenho "típico" das ações.

---

**Detalhamento e Obsevações:**


**Preço de Fechamento Ajustado (Adj Close):** Homogeneidade Clara

- Tanto nos gráficos por Setor quanto por Indústria (Gráficos 1 e 3), as formas dos "violinos" são notavelmente semelhantes. Isso indica que o preço "típico" das ações e a forma como esses preços se espalham são muito consistentes e homogêneos entre diferentes setores e indústrias. Não há um grupo que se destaque significativamente por ter ações fundamentalmente mais caras ou mais baratas que os outros.

**Capitalização de Mercado (Market Cap - USD):** Tamanho Típico Similar, mas com Gigantes Universais

- Nos gráficos de Market Cap (Gráficos 2 e 4), observe que a parte maior de cada violino (representando a maioria das empresas) está posicionada em uma altura muito similar, tanto por setor quanto por indústria. Isso sugere que o valor de mercado "típico" das empresas é bastante parecido, independentemente do grupo ao qual pertencem.
No entanto, todos os violinos apresentam alongamentos longos e finas que se estendem para valores de Market Cap extremamente altos. Isso demonstra que a presença de poucas empresas com capitalizações de mercado gigantescas é uma característica universal no conjunto de dados, aparecendo em praticamente todos os setores e indústrias, e não é um traço exclusivo ou diferenciador de um grupo específico.


**Outras Métricas (PE Ratio e EPS):**

- Embora não ilustrado por gráficos adicionais para manter a concisão, análises anteriores mostraram que PE Ratio e EPS (USD) também apresentam padrões de distribuição muito homogêneos entre setores e indústrias, reforçando a falta de uma associação distintiva.

---

**Conclusão Geral**

Em suma, as métricas de desempenho das ações analisadas (Adj Close, Market Cap, PE Ratio, EPS) demonstraram um comportamento consistentemente homogêneo em suas distribuições e tendências centrais entre os diferentes setores e indústrias. Isso sugere que, para este conjunto de dados, as características setoriais ou industriais não são um fator primário que explique grandes variações no desempenho "típico" das ações.

## **3. Como o volume de negociações e os dividendos distribuídos se comportam entre diferentes setores e ao longo do tempo?**

📊 Setores Analisados

Ao longo do conjunto de dados, três setores são majoritários:

- Technology (Tecnologia)

- Consumer Discretionary (Consumo Discricionário)

- Communication Services (Serviços de Comunicação)


1. Technology (Tecnologia)

📈 Volume de Negociações:



O setor de tecnologia apresenta volumes médios de negociação elevados, com um crescimento contínuo ao longo do tempo, indicando aumento de interesse dos investidores.

Isso pode refletir o crescimento das empresas de tecnologia e o papel dominante que elas passaram a desempenhar nos mercados globais.


💸 Dividendos:



Os dividendos são relativamente baixos e estáveis, o que é típico de setores em crescimento, onde os lucros são reinvestidos.

Pequenas flutuações sugerem ajustes pontuais de política de dividendos, mas a estratégia de retenção de capital prevalece.



---

2. Consumer Discretionary (Consumo Discricionário)

📈 Volume:



O volume é mais moderado em relação à tecnologia, com pequenas oscilações ao longo do tempo.

Isso pode refletir a sazonalidade das empresas desse setor ou ciclos econômicos (como períodos de recessão/expansão do consumo).


💸 Dividendos:



Dividendos ligeiramente mais altos que o setor de tecnologia, mas ainda modestos.

Isso reflete empresas que conciliam reinvestimento com distribuição de lucros — geralmente empresas maduras de bens de consumo.



---

3. Communication Services (Serviços de Comunicação)

📈 Volume:


Volume alto e estável, com ligeiras variações. As empresas deste setor, como redes sociais ou telecomunicações, tendem a ser muito negociadas.

Indica interesse constante do mercado por esses ativos, talvez pela sua volatilidade ou liquidez.


💸 Dividendos:



Dividendos moderados e bastante constantes ao longo dos anos.

Demonstra política de dividendos estável, com empresas já consolidadas que retornam parte dos lucros aos acionistas.




-  Conclusões Gerais

1. Volume de Negociações:

O setor de Tecnologia lidera em volume e crescimento de negociações, mostrando crescente interesse dos investidores.

Serviços de Comunicação também têm alto volume, sugerindo alta liquidez.

Consumo Discricionário é mais moderado, com influência de fatores sazonais e macroeconômicos.



2. Dividendos Distribuídos:

Setores maduros tendem a pagar mais dividendos (caso do Consumo Discricionário e Serviços de Comunicação).

Setores em crescimento, como Tecnologia, priorizam reinvestimento e pagam menos dividendos.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Definir estilo dos gráficos
sns.set(style="whitegrid")

# Criar gráficos de linha para Volume e Dividendos ao longo do tempo por setor
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 10), sharex=True)

# Gráfico de Volume
sns.lineplot(
    data=sector_year_summary,
    x='Year',
    y='Volume',
    hue='Sector',
    marker='o',
    ax=axes[0]
)
axes[0].set_title('Volume Médio de Negociações por Setor ao Longo do Tempo')
axes[0].set_ylabel('Volume Médio')
axes[0].legend(title='Setor', bbox_to_anchor=(1.05, 1), loc='upper left')

# Gráfico de Dividendos
sns.lineplot(
    data=sector_year_summary,
    x='Year',
    y='Dividend',
    hue='Sector',
    marker='s',
    ax=axes[1]
)
axes[1].set_title('Dividendos Médios Distribuídos por Setor ao Longo do Tempo')
axes[1].set_ylabel('Dividendos Médios')
axes[1].legend().remove()

plt.tight_layout()
plt.show()


🔍 Padrões Identificados por Setor:


- 🖥️ Technology (Tecnologia)

📊 Volume:
Crescimento contínuo ao longo do tempo.

Volumes consistentemente altos, sugerindo forte interesse do mercado e crescimento da liquidez no setor.

Pode refletir a ascensão de grandes empresas tech, IPOs, e o aumento da presença no portfólio de investidores.

💰 Dividendos:
Baixos e estáveis.

Segue o padrão de setores em crescimento, que preferem reinvestir lucros a distribuí-los.

Não há grande oscilação, sugerindo política de dividendos conservadora ou foco em valorização de capital.

- 🛒 Consumer Discretionary (Consumo Discricionário)
📊 Volume:
Menor que o de tecnologia, com flutuações moderadas.

Oscilações sugerem sensibilidade ao ciclo econômico — o consumo discricionário tende a variar mais em períodos de crise ou crescimento.

💰 Dividendos:
Levemente maiores que tecnologia, mas ainda modestos.

Algumas empresas do setor já maduras distribuem dividendos, enquanto outras mantêm foco em crescimento.

- 📡 Communication Services (Serviços de Comunicação)
📊 Volume:
Volume consistentemente alto, muito próximo ao da tecnologia.

Reflete o papel central dessas empresas (como redes sociais, telecomunicações) no mercado e sua alta liquidez.

💰 Dividendos:
Dividendos constantes e medianos, sem grandes oscilações.

Indica um perfil híbrido: empresas que cresceram, mas agora operam em um modelo de caixa mais maduro, retornando lucros aos acionistas.







| Setor                  | Volume Médio     | Dividendos Médios  | Padrão Identificado                                      |
| ---------------------- | ---------------- | ------------------ | -------------------------------------------------------- |
| Technology             | Alto e crescente | Baixos e estáveis  | Setor de crescimento, foco em reinvestimento             |
| Consumer Discretionary | Moderado         | Modestos           | Sensível ao ciclo econômico                              |
| Communication Services | Alto             | Altos e constantes | Empresas maduras com políticas de dividendos previsíveis |




In [ ]:
plt.figure(figsize=(10, 6))
avg_dividends_by_sector = df.groupby('Sector')['Dividend'].mean().sort_values(ascending=False)
sns.barplot(x=avg_dividends_by_sector.values, y=avg_dividends_by_sector.index, palette='viridis')
plt.title('Média Geral de Dividendos por Setor')
plt.xlabel('Dividendos Médios')
plt.ylabel('Setor')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[df['Dividend'] > 0], x='Sector', y='Dividend', palette='pastel')
plt.xticks(rotation=45)
plt.title('Distribuição de Dividendos por Setor')
plt.ylabel('Dividendos')
plt.xlabel('Setor')
plt.tight_layout()
plt.show()


### 📈 Padrões Gerais ao Longo do Tempo
- Estabilidade Temporal em Dividendos

Apesar das oscilações econômicas, os dividendos se mantêm surpreendentemente estáveis, indicando resiliência ou previsibilidade nas políticas de distribuição.

- Volume Crescente (principalmente em Tech)

O volume mostra tendência de crescimento suave e progressivo, refletindo maior participação do varejo, ETFs e algoritmos no mercado.

- Setores maduros distribuem mais dividendos

Há uma clara distinção entre setores de crescimento (poucos dividendos) e setores maduros (distribuição consistente).


###🧠 Interpretação Estratégica para Investidores:

Quem busca crescimento de capital tende a olhar para setores como Tecnologia, com alta liquidez e baixo dividendo.

Quem busca renda passiva pode se interessar mais por setores como Serviços de Comunicação, que mantêm dividendos razoáveis e estabilidade de mercado.

A análise de volume também pode indicar interesse institucional, sendo útil para decisões baseadas em fluxo de mercado.



| Perfil de Investidor       | Setor Indicado         | Por Quê                                                                 |
| -------------------------- | ---------------------- | ----------------------------------------------------------------------- |
| **Crescimento**            | Technology             | Alto volume, baixa distribuição — reinvestimento e valorização esperada |
| **Renda passiva**          | Communication Services | Dividendos constantes, bom para geração de caixa                        |
| **Diversificação cíclica** | Consumer Discretionary | Ciclicidade moderada, ideal para compor estratégias balanceadas         |




In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Volume', y='Dividend', hue='Sector', alpha=0.6)
sns.regplot(data=df, x='Volume', y='Dividend', scatter=False, color='black', line_kws={"linewidth":1})
plt.title('Relação entre Volume e Dividendos')
plt.xlabel('Volume')
plt.ylabel('Dividendos')
plt.tight_layout()
plt.show()


🧠 Interpretação para Investidores:

❌ 1. Ausência de Correlação Forte
A linha de regressão é quase horizontal, o que indica uma correlação fraca ou inexistente entre volume de negociação e dividendos.

Ou seja, ações muito negociadas não necessariamente pagam mais dividendos, e vice-versa.

📈 2. Implicações para Estratégias de Investimento

O gráfico mostra claramente que o volume de negociação de uma ação não está fortemente ligado ao quanto ela distribui em dividendos. Isso tem implicações diretas dependendo do tipo de investidor:

- Para investidores focados em renda passiva, como aqueles que vivem de dividendos, o gráfico sugere que eles devem buscar empresas historicamente reconhecidas por boas distribuições — normalmente em setores mais maduros e estáveis, como Serviços de Comunicação. Esses investidores não devem se deixar levar apenas por ações com alto volume de negociação, pois isso não garante bons pagamentos.

- Investidores de curto prazo, como traders ou especuladores, que priorizam liquidez para facilitar entradas e saídas rápidas, devem focar em ações de alto volume, como as do setor de Tecnologia. Porém, devem ter consciência de que essas ações geralmente não oferecem retorno via dividendos, e sim por valorização do preço.

- Investidores com perfil equilibrado, que buscam tanto liquidez quanto algum nível de distribuição, precisam analisar os setores com mais cuidado. O gráfico alerta que não se pode esperar encontrar ambos os atributos (liquidez e dividendos altos) na mesma empresa com facilidade, e que será necessário montar uma carteira diversificada para cobrir essas duas frentes.




Conclusão Prática:

O gráfico quebra um mito comum: ações populares (muito negociadas) não garantem bom retorno em dividendos.
 Investidores devem separar suas análises:

- Liquidez → importante para entrada e saída rápida.

- Dividendos → importante para retorno passivo e longo prazo.

A escolha de ações deve ser alinhada ao perfil de risco e objetivo financeiro do investidor, pois o mercado trata essas duas variáveis de forma bastante independente.